# Imports

In [58]:
import pm4py
import pandas as pd
from TreeToTptp import TreeToTptp
from ProverWrapper import use_provers
from create_thesis_with_2_spec import create_thesis_with_2_spec, LogicFunctions
import os

# Helpers

In [59]:
import networkx as nx
import matplotlib.pyplot as plt
import graphviz
import os


def create_tree_graph(process_tree, output_file="process_tree"):
    if not output_file.endswith('.png'):
        output_file = output_file + '.png'

    dot = graphviz.Digraph()
    dot.attr(rankdir='TB')

    def add_nodes_edges(tree, parent_id=None, counter=[0]):
        current_id = str(counter[0])
        counter[0] += 1

        if tree.children:
            label = str(tree.operator)
            label = 'seq' if label == '->' else 'xor' if label == 'X' else 'loop' if label == '*' else 'and' if label == '+' else label
            dot.node(current_id, label, shape='box')
        else:
            if tree.label is None:
                dot.node(current_id, "", shape='circle',
                         style='filled', fillcolor='black', width='0.3')
            else:
                dot.node(current_id, tree.label, shape='oval')

        if parent_id is not None:
            dot.edge(str(parent_id), current_id)

        for child in tree.children:
            add_nodes_edges(child, int(current_id), counter)

    add_nodes_edges(process_tree)
    dot.render(output_file[:-4], format='png', cleanup=True)

    if not os.path.exists(output_file):
        raise FileNotFoundError(f"Failed to create image file: {output_file}")

    return output_file

In [60]:
def get_vampire_short(vampire: str):
    return ' '.join([l for l in vampire.splitlines() if l.startswith('% Refutation') or l.startswith('% SZS status') or l.startswith('% Termination reason')])

def get_eprover_short(eprover: str):
    return ' '.join([l for l in eprover.splitlines() if l.startswith('# SZS status') or l.startswith('# Proof found') or l.startswith('# No proof found')])

In [61]:
from Julia_Origin import ProcessTreeAdapter, get_pattern_expression, get_results
from converter import convert_to_tptp

def use_julias_code(tree):
    W1 = str(tree)

    W1 = ProcessTreeAdapter.remove_brackets_between_single_quotes(W1)

    W1 = ProcessTreeAdapter.replace_spaces_with_underscore(W1)
    W1 = W1.replace("( ", "(").replace(") ", ")").replace(" )",
                                                          ")").replace("->", ">")

    labelled_pattern_expression = ProcessTreeAdapter.label_expressions(W1)

    pattern_expression = get_pattern_expression(labelled_pattern_expression)

    results = get_results(pattern_expression)[:-1]
    results = [x.rstrip() for x in results.splitlines()]
    tptp = convert_to_tptp(results)
    return pattern_expression, '\n'.join(results), tptp

In [62]:
def run_satisfiability(folder_name, log, threshold, prefix='', use_julia=False):
    try:
        os.makedirs(folder_name, exist_ok=True)
        
        process_tree = pm4py.discover_process_tree_inductive(
            log, threshold, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
        
        if use_julia:
            expr, spec, tptp = use_julias_code(process_tree)
        else:
            expr, spec, tptp = TreeToTptp(prefix).tree_to_tptp(process_tree)
        
        with open(f'{folder_name}/expression.txt', 'w') as f:
            f.write(expr)
        with open(f'{folder_name}/specification.txt', 'w') as f:
            f.write(spec)
        with open(f'{folder_name}/tptp.p', 'w') as f:
            f.write(tptp)

        vampire, eprover = use_provers(f'{folder_name}/tptp.p')
        with open(f'{folder_name}/vampire.txt', 'w') as f:
            f.write(vampire)
        with open(f'{folder_name}/eprover.txt', 'w') as f:
            f.write(eprover)

        create_tree_graph(process_tree, f'{folder_name}/process_tree.png')

        return folder_name, get_vampire_short(vampire), get_eprover_short(eprover)
    except Exception as e:
        print(e)
        return folder_name, 'Error', 'Error'


def run_comparison(folder_name, log, threshold1, threshold2, logic: LogicFunctions, use_julia=False):
    try:
        os.makedirs(folder_name, exist_ok=True)
        
        run_satisfiability(f'{folder_name}/original1', log, threshold1, 'A', use_julia)
        run_satisfiability(f'{folder_name}/original2', log, threshold2, 'B', use_julia)

        result = create_thesis_with_2_spec(f'{folder_name}/original1/tptp.p', f'{folder_name}/original2/tptp.p', logic)
        with open(f'{folder_name}/tptp_{logic.name}.p', 'w') as f:
            f.write(result)

        vampire, eprover = use_provers(f'{folder_name}/tptp_{logic.name}.p')
        with open(f'{folder_name}/vampire_{logic.name}.txt', 'w') as f:
            f.write(vampire)
        with open(f'{folder_name}/eprover_{logic.name}.txt', 'w') as f:
            f.write(eprover)

        return folder_name, get_vampire_short(vampire), get_eprover_short(eprover)
    except Exception as e:
        print(e)
        return folder_name, 'Error', 'Error'


def run_thesis(folder_name, log, threshold, thesis, use_julia=False):
    try:
        os.makedirs(folder_name, exist_ok=True)
        
        run_satisfiability(f'{folder_name}/original', log, threshold, use_julia=use_julia)
        with open(f'{folder_name}/tptp_thesis.p', 'w') as f:
            with open(f'{folder_name}/original/tptp.p', 'r') as f2:
                tptp = f2.read()
                f.write(f'{tptp}\n\n{thesis}')

        vampire, eprover = use_provers(f'{folder_name}/tptp_thesis.p')
        with open(f'{folder_name}/vampire_thesis.txt', 'w') as f:
            f.write(vampire)
        with open(f'{folder_name}/eprover_thesis.txt', 'w') as f:
            f.write(eprover)
            
        return folder_name, get_vampire_short(vampire), get_eprover_short(eprover)
    except Exception as e:
        print(e)
        return folder_name, 'Error', 'Error'

# Logs

In [63]:
running = pm4py.read_xes('Data/running-example.xes')

parsing log, completed traces :: 100%|██████████| 6/6 [00:00<00:00, 1442.58it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home

In [64]:
repair = pm4py.format_dataframe(pd.read_csv("Data/repairExample.csv", sep=','),
                                case_id='Case ID', activity_key='Activity', timestamp_key='Start Timestamp')

/tmp/ipykernel_159510/2258410974.py:1: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  repair = pm4py.format_dataframe(pd.read_csv("Data/repairExample.csv", sep=','),
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13

In [65]:
hospital = pm4py.read_xes('Data/Hospital Billing - Event Log.xes')

parsing log, completed traces :: 100%|██████████| 100000/100000 [00:15<00:00, 6562.82it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=T

In [66]:
bpic = pm4py.read_xes('Data/bpic2012.xes')

parsing log, completed traces :: 100%|██████████| 13087/13087 [00:06<00:00, 2051.41it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=Tru

In [67]:
artificial = pm4py.read_xes('Data/log_3_1732138120.xes')

parsing log, completed traces :: 100%|██████████| 30369/30369 [00:12<00:00, 2513.79it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)


# Theorems

In [68]:
running_thesis = [
    '''
fof(thesis, conjecture, (
    ! [X]: (register_request(X) => ?[Y]: (reject_request(Y) | pay_compensation(Y)))
)).
''',
    '''
fof(thesis, conjecture, (
    ! [X]: (~(reject_request(X) & pay_compensation(X)))
)).
'''
]

repair_thesis = [
    '''
fof(thesis, conjecture, (
    ! [X]: (register(X) => ?[Y]: (repair_simple(Y) | repair_complex(Y)))
)).
''',
    '''
fof(thesis, conjecture, (
    ! [X]: (~(inform_user(X) & tau(X)))
)).
'''
]

# Julia's code

In [69]:
julias_results = [
    run_satisfiability('julia_s_problems/problem1',
                       running, 0.0, use_julia=True),
    run_thesis('julia_s_problems/problem11', running,
               0.0, running_thesis[0], use_julia=True),
    run_thesis('julia_s_problems/problem12', running,
               0.0, running_thesis[1], use_julia=True),

    run_satisfiability('julia_s_problems/problem2',
                       repair, 0.0, use_julia=True),
    run_satisfiability('julia_s_problems/problem3',
                       repair, 0.5, use_julia=True),
    run_satisfiability('julia_s_problems/problem4',
                       repair, 1.0, use_julia=True),
    run_comparison('julia_s_problems/problem5', repair,
                   0.0, 0.5, LogicFunctions.IMPLIES, use_julia=True),
    run_comparison('julia_s_problems/problem6', repair,
                   0.5, 1.0, LogicFunctions.IMPLIES, use_julia=True),
    run_comparison('julia_s_problems/problem8', repair,
                   0.0, 0.5, LogicFunctions.EQUIVALENT, use_julia=True),
    run_comparison('julia_s_problems/problem9', repair,
                   0.5, 1.0, LogicFunctions.EQUIVALENT, use_julia=True),
    run_thesis('julia_s_problems/problem13', repair,
               0.5, repair_thesis[0], use_julia=True),
    run_thesis('julia_s_problems/problem14', repair,
               0.5, repair_thesis[1], use_julia=True),
    run_thesis('julia_s_problems/problem15', repair,
               1.0, repair_thesis[0], use_julia=True),
    run_thesis('julia_s_problems/problem16', repair,
               1.0, repair_thesis[1], use_julia=True),
]

Seq3(1]register_request, Loop(2]l_s,Seq2(3]And2(4]a2_s,check_ticket, Xor2(5]x2_s,examine_thoroughly, examine_casually,x2_e[5),a2_e[4), decide[3), reinitiate_request[2), Xor2(2]x2_s,pay_compensation, reject_request,x2_e[2)[1)
ini: register_request
fin: x2_e

Wynik: 
ForAll(~((examine_thoroughly) ^ (x2_e)))
ForAll(~((reject_request) ^ (x2_e)))
ForAll(~((register_request) ^ (x2_s | x2_e)))
ForAll(~((a2_s) ^ ((check_ticket) | (x2_s | x2_e))))
ForAll((a2_s) => (Exist(check_ticket) ^ Exist(x2_s | x2_e)))
ForAll((reinitiate_request) => Exist(a2_s | decide))
ForAll((x2_s) => ((Exist(pay_compensation) ^ ~(Exist(reject_request))) | (~(Exist(pay_compensation)) ^ Exist(reject_request))))
ForAll(((pay_compensation) | (reject_request)) => Exist(x2_e))
ForAll((x2_s) => ((Exist(examine_thoroughly) ^ ~(Exist(examine_casually))) | (~(Exist(examine_thoroughly)) ^ Exist(examine_casually))))
ForAll(~((pay_compensation) ^ (reject_request)))
ForAll(~((a2_s | decide) ^ (reinitiate_request)))
ForAll(~((x2_s) ^

# Our results

In [70]:
our_results = [
    run_satisfiability('our_problems/problem1',
                       running, 0.0),
    run_thesis('our_problems/problem11', running,
               0.0, running_thesis[0]),
    run_thesis('our_problems/problem12', running,
               0.0, running_thesis[1]),

    run_satisfiability('our_problems/problem2',
                       repair, 0.0),
    run_satisfiability('our_problems/problem3',
                       repair, 0.5),
    run_satisfiability('our_problems/problem4',
                       repair, 1.0),
    run_comparison('our_problems/problem5', repair,
                   0.0, 0.5, LogicFunctions.IMPLIES),
    run_comparison('our_problems/problem6', repair,
                   0.5, 1.0, LogicFunctions.IMPLIES),
    run_comparison('our_problems/problem8', repair,
                   0.0, 0.5, LogicFunctions.EQUIVALENT),
    run_comparison('our_problems/problem9', repair,
                   0.5, 1.0, LogicFunctions.EQUIVALENT),
    run_thesis('our_problems/problem13', repair,
               0.5, repair_thesis[0]),
    run_thesis('our_problems/problem14', repair,
               0.5, repair_thesis[1]),
    run_thesis('our_problems/problem15', repair,
               1.0, repair_thesis[0]),
    run_thesis('our_problems/problem16', repair,
               1.0, repair_thesis[1]),
]

Seq3(1]register_request, Loop(2]l_s_0, Seq2(3]And2(4]a2_s_0, check_ticket, Xor2(5]x2_s_0, examine_thoroughly, examine_casually, x2_e_0[5), a2_e_0[4), decide[3), reinitiate_request[2), Xor2(2]x2_s_1, pay_compensation, reject_request, x2_e_1[2)[1)
ini: register_request
fin: x2_e_1
Seq3(1]register_request, Loop(2]l_s_0, Seq2(3]And2(4]a2_s_0, check_ticket, Xor2(5]x2_s_0, examine_thoroughly, examine_casually, x2_e_0[5), a2_e_0[4), decide[3), reinitiate_request[2), Xor2(2]x2_s_1, pay_compensation, reject_request, x2_e_1[2)[1)
ini: register_request
fin: x2_e_1
Seq3(1]register_request, Loop(2]l_s_0, Seq2(3]And2(4]a2_s_0, check_ticket, Xor2(5]x2_s_0, examine_thoroughly, examine_casually, x2_e_0[5), a2_e_0[4), decide[3), reinitiate_request[2), Xor2(2]x2_s_1, pay_compensation, reject_request, x2_e_1[2)[1)
ini: register_request
fin: x2_e_1
Seq5(1]register, analyze_defect, And2(2]a2_s_0, Xor2(3]x2_s_0, tau_0, inform_user, x2_e_0[3), And2(3]a2_s_1, Xor2(4]x2_s_1, tau_1, Loop(5]l_s_0, test_repair, ta

In [71]:
for r in julias_results:
    print(r[0])
    print('  Vampire:')
    for i in r[1].split('%')[1:]:
        print(f'    %{i}')
    print('  Eprover:')
    for i in r[2].split('#')[1:]:
        print(f'    %{i}')
    print('\n')

julia_s_problems/problem1
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status Satisfiable


julia_s_problems/problem11
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_thesis 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Theorem


julia_s_problems/problem12
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_thesis 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Theorem


julia_s_problems/problem2
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Unsatisfiable for tptp 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Unsatisfiable


julia_s_problems/problem3
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status S

In [72]:
for r in our_results:
    print(r[0])
    print('  Vampire:')
    for i in r[1].split('%')[1:]:
        print(f'    %{i}')
    print('  Eprover:')
    for i in r[2].split('#')[1:]:
        print(f'    %{i}')
    print('\n')

our_problems/problem1
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status Satisfiable


our_problems/problem11
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_thesis 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Theorem


our_problems/problem12
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_thesis 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Theorem


our_problems/problem2
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Unsatisfiable for tptp 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Unsatisfiable


our_problems/problem3
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status Satisfiable


our_pro

In [73]:
for r in julias_results:
    print(f'{r[0]},{r[1]},{r[2]}')

julia_s_problems/problem1,% SZS status Satisfiable for tptp % Termination reason: Satisfiable,# No proof found! # SZS status Satisfiable
julia_s_problems/problem11,% Refutation found. Thanks to Tanya! % SZS status Theorem for tptp_thesis % Termination reason: Refutation,# Proof found! # SZS status Theorem
julia_s_problems/problem12,% Refutation found. Thanks to Tanya! % SZS status Theorem for tptp_thesis % Termination reason: Refutation,# Proof found! # SZS status Theorem
julia_s_problems/problem2,% Refutation found. Thanks to Tanya! % SZS status Unsatisfiable for tptp % Termination reason: Refutation,# Proof found! # SZS status Unsatisfiable
julia_s_problems/problem3,% SZS status Satisfiable for tptp % Termination reason: Satisfiable,# No proof found! # SZS status Satisfiable
julia_s_problems/problem4,% Refutation found. Thanks to Tanya! % SZS status Unsatisfiable for tptp % Termination reason: Refutation,# Proof found! # SZS status Unsatisfiable
julia_s_problems/problem5,% Refutation

In [74]:
for r in our_results:
    print(f'{r[0]},{r[1]},{r[2]}')

our_problems/problem1,% SZS status Satisfiable for tptp % Termination reason: Satisfiable,# No proof found! # SZS status Satisfiable
our_problems/problem11,% Refutation found. Thanks to Tanya! % SZS status Theorem for tptp_thesis % Termination reason: Refutation,# Proof found! # SZS status Theorem
our_problems/problem12,% Refutation found. Thanks to Tanya! % SZS status Theorem for tptp_thesis % Termination reason: Refutation,# Proof found! # SZS status Theorem
our_problems/problem2,% Refutation found. Thanks to Tanya! % SZS status Unsatisfiable for tptp % Termination reason: Refutation,# Proof found! # SZS status Unsatisfiable
our_problems/problem3,% SZS status Satisfiable for tptp % Termination reason: Satisfiable,# No proof found! # SZS status Satisfiable
our_problems/problem4,% Refutation found. Thanks to Tanya! % SZS status Unsatisfiable for tptp % Termination reason: Refutation,# Proof found! # SZS status Unsatisfiable
our_problems/problem5,% Refutation found. Thanks to Tanya! % S